
*   *Подключение библиотек:*

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 


*   *Подключение начальных данных*

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Home_price /train.csv')
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,14,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,3,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,3,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,5,336,670,869,18,10,19,1,1,1,0


In [ ]:
corrmat = df.corr()
corrmat['price_range']

corrmat['price_range'] = abs(corrmat['price_range'])
new = corrmat.sort_values(by=['price_range']) 
new['price_range']  

m_dep            0.000853
n_cores          0.004399
clock_speed      0.006606
four_g           0.014772
dual_sim         0.017444
wifi             0.018785
blue             0.020573
talk_time        0.021859
fc               0.021998
sc_h             0.022986
three_g          0.023611
mobile_wt        0.030302
touch_screen     0.030411
pc               0.033599
sc_w             0.038711
int_memory       0.044435
px_height        0.148858
px_width         0.165818
battery_power    0.200723
ram              0.917046
price_range      1.000000
Name: price_range, dtype: float64

In [ ]:
features = ['ram', 'battery_power', 'px_width', 'px_height', 'int_memory', 'sc_w', 'pc', 'touch_screen',
            'mobile_wt', 'three_g', 'sc_h', 'price_range']
df = df[features]
df

,ram,battery_power,px_width,px_height,int_memory,sc_w,pc,touch_screen,mobile_wt,three_g,sc_h,price_range
0,2549,842,756,20,7,7,2,0,188,0,9,1
1,2631,1021,1988,905,53,3,6,1,136,1,17,2
2,2603,563,1716,1263,41,2,6,1,145,1,11,2
3,2769,615,1786,1216,10,8,9,0,131,1,16,2
4,1411,1821,1212,1208,44,2,14,1,141,1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,668,794,1890,1222,2,4,14,1,106,1,13,0
1996,2032,1965,1965,915,39,10,3,1,187,1,11,2
1997,3057,1911,1632,868,36,1,3,1,108,1,9,3
1998,869,1512,670,336,46,10,5,1,145,1,18,0


In [ ]:
X = df.drop(['price_range'], axis=1)
y = df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)
df_train = X_train
df_train = df_train.join(y_train)
df_test = X_test
df_test = df_test.join(y_test)

In [ ]:
  class DecisionTree:

    # Инициализация класса
    def __init__(self, df):
      self.header = np.array(df.columns)
      self.rows = np.array(df)
      self.tree = None
    
    # уникальные значения
    def unique_vals(self, rows, col):
      return set([row[col] for row in rows])

    # подсчет классов
    def class_counts(self,rows):
      counts = {} 
      for row in rows:
        label = row[-1]
        if label not in counts:
          counts[label] = 0
        counts[label] += 1
      return counts
    
    # Проверка является ли значение числом
    def is_numeric(value):
      return isinstance(value, np.int64) or isinstance(value, float)

    class Question():

      def __init__(self, column, value):
        self.column = column
        self.value = value

      def match(self, example):
        val = example[self.column]
        if DecisionTree.is_numeric(val):
          return val >= self.value
        else:
          return val == self.value

    # Разделение на правильные ответы и нет
    def partition(self, rows, question):
      true_rows, false_rows = [], []
      for row in rows:
        if question.match(row):
          true_rows.append(row)
        else:
          false_rows.append(row)
      return true_rows, false_rows
    
    # Подсчет индекса Джини
    def gini(self, rows):
      counts = self.class_counts(rows)
      impurity = 1
      for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
      return impurity

    # Подсчет прироста информации
    def info_gain(self, left, right, current_uncertainty):
      p = float(len(left)) / (len(left) + len(right))
      return current_uncertainty - p * self.gini(left) - (1 - p) * self.gini(right)

    # Поиск лучшего разделения на узлы
    def find_best_split(self, rows):
      best_gain = 0  
      best_question = None  
      current_uncertainty = self.gini(rows)
      n_features = len(rows[0]) - 1  

      for col in range(n_features):  

          values = set([row[col] for row in rows])  

          for val in values:  

              question = self.Question(col, val)

              true_rows, false_rows = self.partition(rows, question)

              if len(true_rows) == 0 or len(false_rows) == 0:
                  continue

              gain = self.info_gain(true_rows, false_rows, current_uncertainty)

              if gain >= best_gain:
                  best_gain, best_question = gain, question

      return best_gain, best_question

    class Leaf():
      def __init__(self, DecisionTree, rows ):
          self.predictions = DecisionTree.class_counts(rows)
    
    class Decision_Node():
      def __init__(self, question, true_branch, false_branch):
          self.question = question
          self.true_branch = true_branch
          self.false_branch = false_branch

    # Построение дерева
    def build_tree(self, rows):
      gain, question = self.find_best_split(rows)

      if gain == 0:
          return self.Leaf(self, rows)

      true_rows, false_rows = self.partition(rows, question)

      true_branch = self.build_tree(true_rows)

      false_branch = self.build_tree(false_rows)

      return self.Decision_Node(question, true_branch, false_branch)

    # Обучение
    def fit(self, train_data):
      self.tree = self.build_tree(train_data)

    # Предсказание
    def classify(self, row, node):
      if isinstance(node, self.Leaf):
          return node.predictions
      if node.question.match(row):
          return self.classify(row, node.true_branch)
      else:
          return self.classify(row, node.false_branch)


In [ ]:
my_tree = DecisionTree(df_train)
my_tree.fit(np.array(df_train))

In [ ]:
y_pr = []
for row in np.array(df_test):
    y_pr.append((list(my_tree.classify(np.array(row), my_tree.tree))[0]))

accuracy_score(y_pr, y_test)*100

83.625

In [ ]:
features = ['ОЗУ(в Мегабайтах) ', 'Батарея(mAh) ', 'Разрешение экрана (высота)', 'Разрешение экрана (ширина)', 'Внутренняя память (в Гигабайтах)', 'Высота экрана (в см)', 'Разрешение основной камеры (в МегаПикселях)', 'Сенсорный экран (да - 1, нет - 0)',
            'Вес телефона (в граммах)', 'Есть ли 3G (да - 1, нет - 0)', 'Ширина экрана (в см)', 'price_range']

In [ ]:
test = []
for i in features[:-1]:
  feature = int(input(i + ": "))
  test.append(feature)

test.append(0)

ОЗУ(в Мегабайтах) : 512
Батарея(mAh) : 2
Разрешение экрана (высота): 50
Разрешение экрана (ширина): 60
Внутренняя память (в Гигабайтах): 2
Высота экрана (в см): 10
Разрешение основной камеры (в МегаПикселях): 2
Сенсорный экран (да - 1, нет - 0): 0
Вес телефона (в граммах): 200
Есть ли 3G (да - 1, нет - 0): 0
Ширина экрана (в см): 5


In [ ]:
list(my_tree.classify(np.array(test), my_tree.tree))[0]

0